In [44]:
import sys
dll_path = r'C:\Users\bkmz1\PycharmProjects\manipulator\dobot_lib'
sys.path.insert(1,dll_path)
import DobotDllType as dType
from math import pi
import numpy as np
import time
import scipy.optimize as optimize
from math import dist, atan2
from itertools import combinations, islice
import pickle
import ipywidgets as widgets

In [2]:
with open('coords.pickle', 'rb') as f:
    data = pickle.load(f)

In [55]:
dir(db)

['R_ab',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'api',
 'board_xy_from_px',
 'calibrate_height',
 'calibrate_image',
 'calibrate_pos',
 'change_j4',
 'chess_piece_height',
 'copy',
 'copy_data',
 'dobot_xy_from_board_xy',
 'get_height',
 'get_xyz',
 'move_new_z',
 'move_piece',
 'move_to_px',
 'move_to_tile',
 'move_xyz',
 'offset',
 'origin',
 'pieces_heights',
 'read_coords',
 'temp',
 'tile_size',
 'zero_height']

In [58]:
print(db.offset, db.origin, db.pieces_heights, db.temp, db.tile_size, db.zero_height, sep='\n')

0
[13.70218526  6.29568487]
{'pawn': -45.83985900878906, 'rook': -40.83985900878906, 'knight': -40.83985900878906, 'bishop': -40.83985900878906, 'queen': -40.83985900878906, 'king': -14.503204345703125}
{'bc': ['d8', 'h4', 'e2'], 'dc': [[292.9919128417969, -41.000938415527344], [189.7934112548828, 76.61956024169922], [278.457275390625, 126.46517944335938]], 'j1': [-7.9661712646484375, 21.983919143676758, 24.42582130432129]}
[27.58505335]
-60.57623291015625


In [3]:
com = 'COM4'
api = dType.load(dll_path)

The dll you are using is 64-bit. In order to run smoothly, please ensure that your python environment is also 64-bit.
The python environment is： ('64bit', 'WindowsPE')


In [4]:
state = dType.ConnectDobot(api, "", 115200)[0]
print("Connect status:", state)

dType.SetQueuedCmdClear(api)
dType.SetQueuedCmdStartExec(api)

dType.SetHOMEParams(api, 200, 200, 200, 200, isQueued = 1)
dType.SetPTPJointParams(api, 200, 200, 200, 200, 200, 200, 200, 200, isQueued = 1)
dType.SetPTPCommonParams(api, 100, 100, isQueued = 1)

Connect status: 0


[4]

In [144]:
dType.DisconnectDobot(api)

In [39]:
def coords_from_chess_pos(pos):
    # from chess pos (e.g. 'e4') into local chess coords
    # letters - xi axis, numbers - eta. Origin at center of a1
    assert len(pos) == 2
    if isinstance(pos, str):
        char, num = pos.lower()
        xi = ord(char) - ord('a')
        eta = int(num) - 1
        return np.array([xi, eta])
    else:
        return np.array(pos)
    

def find_intersections(o1, r1, o2, r2, eps=.1):
    # finds intersection points of 2 circles
    # if no intersection, raises exception
    o1, o2 = numpify(o1, o2)
    d = dist(o1, o2)
    assert d <= r1+r2
    assert d > eps
    a = (r1**2-r2**2+d**2)/(2*d)
    h = (r1**2 - a**2)**.5
    P = (o2-o1)*a/d+o1
    perp_ = perp(o2-o1)
    I1, I2 = P+h*perp_, P-h*perp_
    return I1, I2

def normalize(vec):
    return np.array(vec)/length(vec)

def perp(vec):
    return normalize(vec)[::-1]*np.array([-1,1])

def length(vec):
    return dist(vec, [0]*len(vec))

def numpify(*args):
    arrays = [0]*len(args)
    for i, arg in enumerate(args):
        arrays[i] = arg.astype(np.float32) if isinstance(arg, np.ndarray) else np.array(arg, dtype=np.float32)
    return arrays

def find_angle(vec_a, vec_b):
    """
    finds angle from vec_b to vec_a in [0; 2pi) (counterclockwise)
    """
    return (atan2(*vec_a)-atan2(*vec_b))%(2*pi)

def find_circles(A_b, B_b, A_d, B_d):
    # find center and radius of circle, given coords in board frame and dobot frame
    # returns o1, o2, r
    A_b, B_b, A_d, B_d = numpify(A_b, B_b, A_d, B_d)
    a = length(A_d)
    b = length(B_d)
    c = length(A_d - B_d)
    cos_alpha = (a**2+b**2-c**2)/2/a/b
    sin_alpha = (1-cos_alpha**2)**.5
    AB_b = B_b-A_b
    l = length(AB_b)
    r = l/2/sin_alpha
    normal = r*cos_alpha*perp(AB_b)
    M = (A_b+B_b)/2
    o1, o2 = M+normal, M-normal
    return o1, o2, r

def count_coincides(A, points, eps = .1):
    return len([0 for point in points if dist(A, point) < eps])

def is_clockwise(vec1, vec2):
    # finds if vec2 is rotated clockwise from vec1
    cross = vec1[0]*vec2[1] - vec1[1]*vec2[0]
    assert cross != 0
    return cross < 0

def find_origin(board_coords, dobot_coords, j1_positions):
    # find center, given coords of 3 points
    # points all have different angle for 1st joint
    assert len(board_coords) == len(dobot_coords) == 3
    circles = []
    board_coords = list(map(coords_from_chess_pos, board_coords))
    for i, j in combinations([0,1,2], 2):
        o1, o2, r = find_circles(board_coords[i], board_coords[j], dobot_coords[i], dobot_coords[j])
        circles.extend(([o1, r], [o2, r]))
    inters = []
    for circ1, circ2 in combinations(circles, 2):
        inters.extend(find_intersections(*circ1, *circ2))
    centers = [p for p in inters if count_coincides(p, inters) == 3]
    c0 = centers[0]
    eps = .1
    for c in centers:
        if dist(c0, c) > eps:
            c1 = c
            break
    board_coords.append(board_coords[0])
    j1_positions = j1_positions + [j1_positions[0]]
    for center in (c0, c1):
        vectors = [b_point - center for b_point in board_coords]
        for v1, v2, j_val1, j_val2 in zip(vectors[:-1], vectors[1:], j1_positions[:-1], j1_positions[1:]):
            # clockwise rotation is changing j1 angle to lesser angle
            if not is_clockwise(v1, v2) == (j_val2 - j_val1 < 0):
                break
        else:
            return center
    raise ValueError('bad points passed')

def find_rot_angle(p_b, q_a, origin):
    """
    returns rotation matrix R_ab
    p_b - coordinates of point in board frame
    q_a - coordinates shown by dobot
    origin - coordinates of dobot's origin in board frame
    """
    p_b = coords_from_chess_pos(p_b)
    p_b, q_a, origin = numpify(p_b, q_a, origin)
    # same vector in different frames
    v_b = normalize(p_b-origin)
    v_a = normalize(q_a)
    
    angle = find_angle(v_b, v_a)
    return angle

def find_l(board_coords, dobot_coords, origin):
    """
    board_coords - coords of 3 points in board's frame
    dobot_coords - coords shown by dobot
    origin - origin of dobot in board's frame
    
    returns d - offset, l - size of tile (both in mm, if given dobot coords are in mm)
    """
    board_coords = list(map(coords_from_chess_pos, board_coords))
    dobot_coords, origin = numpify(dobot_coords, origin)
    lengths_b = [length(point-origin) for point in board_coords]
    lengths_a = [length(point) for point in dobot_coords]
    def la_from_lb(l_b, l):
        return l*l_b
    popt, _ = optimize.curve_fit(la_from_lb, lengths_b, lengths_a, [28])
    print('d, l optimization error:', sum([abs(la_from_lb(l_b, *popt) - l_a) for l_b, l_a in zip(lengths_b, lengths_a)]))
    return popt

def rot_matr(phi):
    c, s = np.cos(phi), np.sin(phi)
    return np.array([[c, -s],
                     [s, c]])

In [6]:
def calibrate_by_optimizing(board_xy, dobot_xy, defaults=None):
    def get_coords(x):
        phi, o_x, o_y, tile_size, offset = x
        summ = 0
        for p_b, p_a in zip(board_xy, dobot_xy):
            c, s = np.cos(phi), np.sin(phi)
            R_ab = np.array([[c, -s],
                             [s, c]])
            origin = np.array([o_x, o_y])
            t = R_ab@(coords_from_chess_pos(p_b) - origin)
            p_a_calc = t*tile_size - offset * normalize(t)
            summ += dist(p_a, p_a_calc)
        return summ
    if defaults is None:
        defaults = [3.14/2,4, 16, 28, -30]
    optimizer = optimize.minimize(get_coords, defaults,  method='Nelder-Mead', options={'maxiter':10**10})
    phi, o_x, o_y, tile_size, offset = optimizer.x
    R_ab = rot_matr(phi)
    print(f'error distance is: {get_coords(optimizer.x)}')
    return R_ab, np.array([o_x, o_y]), tile_size, offset

def calibrate_by_optimizing_2(board_xy, dobot_xy, tile_size):
    def get_coords(x):
        phi, o_x, o_y, offset = x
        summ = 0
        for p_b, p_a in zip(board_xy, dobot_xy):
            c, s = np.cos(phi), np.sin(phi)
            R_ab = np.array([[c, -s],
                             [s, c]])
            origin = np.array([o_x, o_y])
            t = R_ab@(coords_from_chess_pos(p_b) - origin)
            p_a_calc = t*tile_size - offset * normalize(t)
            summ += dist(p_a, p_a_calc)
        return summ
    optimizer = optimize.minimize(get_coords, [3.14/2, 4, 16, 0],  method='Nelder-Mead', options={'maxiter':10**10})
    phi, o_x, o_y, offset = optimizer.x
    c, s = np.cos(phi), np.sin(phi)
    R_ab = rot_matr(phi)
    print(f'error distance is: {get_coords(optimizer.x)}, {offset=}')
    return R_ab, np.array([o_x, o_y]), offset

In [40]:
class Dobot:
    def __init__(self, api, copy_from=None, from_data=None):
        """
        origin - dobot origin in board's coords
        """
        self.api = api
        self.temp = {}
        self.zero_height = None
        self.pieces_heights = {}
        if copy_from is not None:
            self.copy(copy_from)
        elif from_data is not None:
            self.copy_data(data)
            
    
    def copy(self, other_dobot):
        self.temp = other_dobot.temp
        try:
            self.calibrate_pos(0)
        except Exception as e:
            print('exception, while trying to calibrate')
        self.zero_height = other_dobot.zero_height
        self.pieces_heights = other_dobot.pieces_heights
        
    def copy_data(self, data):
        self.temp = data['temp']
        self.zero_height = data['zero_height']
        self.pieces_heights = data['heights']
    
    def dobot_xy_from_board_xy(self, board_xy):
        """
        board_xy - can either be like 'b3' or like [1, 2]
        """
        t = self.R_ab@(coords_from_chess_pos(board_xy) - self.origin)
        return t*self.tile_size - self.offset * normalize(t)
    
    def move_xyz(self, x, y, z, j4_rot=0):
        dType.SetPTPCmd(self.api,
            dType.PTPMode.PTPMOVLXYZMode,
            x,
            y,
            z,
            j4_rot,
            isQueued=1
        )
    
    def move_new_z(self, z):
        self.move_xyz(*dType.GetPose(self.api)[:2], z, dType.GetPose(self.api)[3])
    
    def change_j4(self, j4):
        self.move_xyz(*dType.GetPose(self.api)[:3], j4)
    
    def chess_piece_height(self, piece):
        print(piece)
        default_offset = 20
        if piece is None:
            return self.zero_height + default_offset
        else:
            return self.pieces_heights[piece]
    
    def move_to_tile(self, board_xy, chess_piece=None, height=None, j4_rot=0):
        dobot_xy = self.dobot_xy_from_board_xy(board_xy)
        if height is None:
            height = self.chess_piece_height(chess_piece)
        self.move_xyz(*dobot_xy, height, j4_rot)
    
    def get_height(self):
        return dType.GetPose(self.api)[2]
    
    def get_xyz(self):
        return np.array(dType.GetPose(self.api)[:3])
    
    def calibrate_height(self):
        input('Move to zero height')
        self.zero_height = self.get_height()
        pieces = ['pawn', 'rook', 'knight', 'bishop', 'queen', 'king']
        for piece in pieces:
            input(f"Move to {piece}'s height")
            self.pieces_heights[piece] = self. get_height()
            print(f'Height is {self.pieces_heights[piece]}')
    
    def read_coords(self, num_of_points=3, rewrite_coords=True):
        board_coords = []
        dobot_coords = []
        j1_poses = []
        if not rewrite_coords:
            board_coords, dobot_coords, j1_poses = self.temp['bc'], self.temp['dc'], self.temp['j1']
        delta = widgets.RadioButtons(options=['0.1 mm', '1 mm', '10 mm'],)
        change_coords = [
            widgets.Button(description='-X'),
            widgets.Button(description='+X'),
            widgets.Button(description='-Y'),
            widgets.Button(description='+Y'),
            widgets.Button(description='-Z'),
            widgets.Button(description='+Z'),
        ]
        record = widgets.Button(description='record')
        inp = widgets.Text()
        def stop_recording():
            record.disabled = True
            for button in change_coords:
                button.disabled = True
            self.temp['bc'] = board_coords
            self.temp['dc'] = dobot_coords
            self.temp['j1'] = j1_poses
            print('stopped recording!')
            
        temp = {'num_of_points':num_of_points}
        def on_click_record(b):
            if inp.value == '':
                stop_recording()
            else:
                board_coords.append(inp.value)
                inp.value = ''
                dobot_coords.append(dType.GetPose(self.api)[:2])
                j1_poses.append(dType.GetPose(self.api)[4])
                temp['num_of_points'] -= 1
                if not temp['num_of_points']:
                    stop_recording()
        def change_coord_on_click(b):
            mask = {
                'X': [1, 0, 0],
                'Y': [0, 1, 0],
                'Z': [0, 0, 1]
            }[b.description[1]]
            mask = np.array(mask, dtype=float)
            if b.description[0] == '-':
                mask *= -1
            mask *= [.1, 1, 10][delta.get_state()['index']]
            self.move_xyz(*(self.get_xyz() + mask))
        
        record.on_click(on_click_record)
        for button in change_coords:
            button.on_click(change_coord_on_click)
        display(delta, *change_coords, record, inp)
        
        
    
    def calibrate_pos(self, method=1, ind_start=0, ind_end=3):
        board_coords, dobot_coords, j1_poses = self.temp['bc'], self.temp['dc'], self.temp['j1']
        origin = find_origin(board_coords[ind_start:ind_end], dobot_coords[ind_start:ind_end], j1_poses[ind_start:ind_end])
        phi = find_rot_angle(board_coords[0], dobot_coords[0], origin)
        offset = 0
        tile_size = find_l(board_coords, dobot_coords, origin)
        #dist(db.temp['dc'][0], db.temp['dc'][-1])/dist(*map(coords_from_chess_pos, (db.temp['bc'][0], db.temp['bc'][-1])))
        if method == 0:
            self.origin = origin
            self.R_ab = rot_matr(phi)
            self.offset, self.tile_size = offset, tile_size
        elif method in [1, 2]:
            defaults = [phi, *origin, tile_size, offset]
            if method == 1:
                self.R_ab, self.origin, self.tile_size, self.offset = calibrate_by_optimizing(board_coords, dobot_coords)
            if method == 2:
                self.tile_size = tile_size
                self.R_ab, self.origin, self.offset = calibrate_by_optimizing_2(board_coords, dobot_coords, self.tile_size)
        print(self.R_ab, self.origin, self.tile_size, self.offset, sep='\n')
    
    def calibrate_image(self, tile_size_px, image_center_px, bottom):
        """
        image_center_px - coords of center of the image. It is half tile lower than board's center
        bottom - where row, where white stand, is
            S, E, N, W
        """
        self.tile_size_px = tile_size_px
        self.image_center_px = np.array(image_center_px)
        if bottom == 'S':
            self.coords_changer = np.array([[1, 0],
                                            [0, -1]])
        if bottom == 'N':
            self.coords_changer = np.array([[-1, 0],
                                            [0, 1]])
        if bottom == 'E':
            self.coords_changer = np.array([[0, -1],
                                            [-1, 0]])
        if bottom == 'W':
            self.coords_changer = np.array([[0, 1],
                                            [1, 0]])
        
    def board_xy_from_px(self, px_xy):
        board_delta = (np.array(px_xy) - self.image_center_px)/self.tile_size_px
        board_delta[1] += .5
        board_coords = self.coords_changer@board_delta + np.array([3.5, 3.5])
        
    def move_to_px(self, px_xy, piece=None):
        self.move_to_tile(self.board_xy_from_px(px_xy), piece)
    
    def move_piece(self, from_coords, to_coords, piece):
        trans_height = (max(self.pieces_heights.values()) - self.zero_height)*2+20 + self.zero_height
        self.move_new_z(trans_height)
        self.move_to_tile(from_coords, height=trans_height)
        self.move_to_tile(from_coords, height=self.chess_piece_height(piece))
        dType.SetEndEffectorSuctionCup(api, 1, 1, True)
        time.sleep(3)
        
        self.move_to_tile(from_coords, height=trans_height)
        self.move_to_tile(to_coords, height=trans_height)
        self.move_to_tile(to_coords, height=self.chess_piece_height(piece))
        dType.SetEndEffectorSuctionCup(api, 1, 0, True)
        time.sleep(1)

In [41]:
db = Dobot(api, copy_from=db)

d, l optimization error: 2.282286066675681
[[-0.99731327 -0.0732546 ]
 [ 0.0732546  -0.99731327]]
[13.70218526  6.29568487]
[27.58505335]
0


In [10]:
db.read_coords(-1)

RadioButtons(options=('0.1 mm', '1 mm', '10 mm'), value='0.1 mm')

Button(description='-X', style=ButtonStyle())

Button(description='+X', style=ButtonStyle())

Button(description='-Y', style=ButtonStyle())

Button(description='+Y', style=ButtonStyle())

Button(description='-Z', style=ButtonStyle())

Button(description='+Z', style=ButtonStyle())

Button(description='record', style=ButtonStyle())

Text(value='')

stopped recording!


In [13]:
dist(db.temp['dc'][0], db.temp['dc'][-1])/dist(*map(coords_from_chess_pos, (db.temp['bc'][0], db.temp['bc'][-1])))

27.63475891784952

In [14]:
dist(db.temp['dc'][0], db.temp['dc'][-1])/6

28.015946013219935

In [91]:
db.temp

{'bc': ['e8', 'g7', 'd4', 'e2'],
 'dc': [[267.1821594238281, -24.27154541015625],
  [210.69607543945312, 5.0494513511657715],
  [296.0400695800781, 87.86585235595703],
  [269.6284484863281, 145.03778076171875]],
 'j1': [-5.190655708312988,
  1.3728630542755127,
  16.53109359741211,
  28.276569366455078]}

In [23]:
dType.GetEndEffectorParams(api)

[59.70000076293945, 0.0, 0.0]

In [43]:
for i in range(3):
    vec = db.dobot_xy_from_board_xy(db.temp['bc'][i])
    print(vec)
    print(vec*(length(db.temp['dc'][i])/length(vec)))
    print(length(db.temp['dc'][i]) - length(vec))
    print()

[293.00394265 -41.00261997]
[292.9919131  -41.00093657]
-0.012146768922946194

[191.0431116  77.1240676]
[189.79341002  76.61956331]
-1.3476938230176643

[277.61739522 126.08375106]
[278.45726985 126.46519165]
0.9224349886359846



In [15]:
db.calibrate_pos(method=0)

[[-0.99731327 -0.0732546 ]
 [ 0.0732546  -0.99731327]]
[13.70218526  6.29568487]
28.22
0


In [20]:
data['temp'] = {'bc': ['c4', 'h3', 'h7', 'a6', 'b5'],
 'dc': [[259.0032653808594, -46.066925048828125],
  [284.808349609375, 94.43470001220703],
  [177.1175994873047, 97.26470184326172],
  [204.3495330810547, -105.91143798828125],
  [231.33233642578125, -75.40081787109375]],
 'j1': [-10.085293769836426,
  18.344118118286133,
  28.773529052734375,
  -27.397058486938477,
  -18.052940368652344]}

In [42]:
db.move_piece('e2', 'e4', 'pawn')
input()
db.move_piece('e7', 'e5', 'pawn')
input()
db.move_piece('e1', 'e2', 'king')
input()
db.move_piece('e8', 'e7', 'king')

pawn
pawn

pawn
pawn

king
king

king
king


In [38]:
db.move_piece('e8', 'e7', 'king')

king
king


In [47]:
db.temp

{'bc': ['e8', 'g7', 'd4', 'h1'],
 'dc': [[266.4316101074219, -17.031978607177734],
  [212.35418701171875, 9.959300994873047],
  [295.51904296875, 88.67515563964844],
  [187.85848999023438, 170.5806427001953]],
 'j1': [-3.6577281951904297,
  2.6851744651794434,
  16.702701568603516,
  42.24030685424805]}

In [48]:
with open('coords.pickle', 'wb') as f:
    pickle.dump({'temp':db.temp, 'zero_height':db.zero_height, 'heights':db.pieces_heights}, f)

In [46]:
positions = []
while input() != 'q':
    positions.append(dType.GetPose(api))
with open('positions_to_check', 'wb') as f:
    pickle.dump({'coords':positions, 'offset': dType.GetEndEffectorParams(api)}, f)










q
